# Burst Image Creation
The foundational data for the burst list is generated by [Christian Monstein](https://www.e-callisto.org/contact.html). The explanation about the types can be found [here](https://e-callisto.org/eventSummaryExplanation.pdf). This base data provides the initial set of sunburst events, which are then refined and corrected using the following [notebook](https://github.com/i4Ds/Radio-sunburst-labelcreation). It performs several key functions:

1. **Typos Correction**: Fixes typographical errors in the original data.
2. **Time Validation**: Removes entries with impossible or incorrect time values.
3. **List Creation**: Compiles a cleaned and corrected list of sunburst events.

With the curated burst list ready, one can then iterate through it to generate visual plots.

To load the included burst list, use the following code:

```python
from ecallisto_ng.burst_list.utils import load_burst_list
burst_list = load_burst_list()
burst_list.head()

|     date | time        |   type | instruments       | time_start   | time_end   |   date_start |   date_end | datetime_start      | datetime_end        |
|---------:|:------------|-------:|:------------------|:-------------|:-----------|-------------:|-----------:|:--------------------|:--------------------|
| 20210119 | 02:42-02:42 |      3 | Australia-ASSA    | 02:42        | 02:42      |     20210119 |   20210119 | 2021-01-19 02:42:00 | 2021-01-19 02:42:00 |
| 20210120 | 12:37-12:37 |      3 | AUSTRIA-UNIGRAZ   | 12:37        | 12:37      |     20210120 |   20210120 | 2021-01-20 12:37:00 | 2021-01-20 12:37:00 |
| 20210120 | 12:37-12:37 |      3 | HUMAIN            | 12:37        | 12:37      |     20210120 |   20210120 | 2021-01-20 12:37:00 | 2021-01-20 12:37:00 |
| 20210120 | 12:37-12:37 |      3 | MRT1              | 12:37        | 12:37      |     20210120 |   20210120 | 2021-01-20 12:37:00 | 2021-01-20 12:37:00 |
| 20210120 | 12:37-12:37 |      3 | SOUTHAFRICA-SANSA | 12:37        | 12:37      |     20210120 |   20210120 | 2021-01-20 12:37:00 | 2021-01-20 12:37:00 |
```
You can also pass a path to a custom burst list file to the `load_burst_list` function.
```python
burst_list = load_burst_list('path/to/burst_list.xlsx')
```

In [1]:
from ecallisto_ng.burst_list.utils import load_burst_list
burst_list = load_burst_list()
burst_list.head()

,date,time,type,instruments,time_start,time_end,date_start,date_end,datetime_start,datetime_end
0,20210119,02:42-02:42,3,Australia-ASSA,02:42,02:42,20210119,20210119,2021-01-19 02:42:00,2021-01-19 02:42:00
1,20210120,12:37-12:37,3,AUSTRIA-UNIGRAZ,12:37,12:37,20210120,20210120,2021-01-20 12:37:00,2021-01-20 12:37:00
2,20210120,12:37-12:37,3,HUMAIN,12:37,12:37,20210120,20210120,2021-01-20 12:37:00,2021-01-20 12:37:00
3,20210120,12:37-12:37,3,MRT1,12:37,12:37,20210120,20210120,2021-01-20 12:37:00,2021-01-20 12:37:00
4,20210120,12:37-12:37,3,SOUTHAFRICA-SANSA,12:37,12:37,20210120,20210120,2021-01-20 12:37:00,2021-01-20 12:37:00


## Automatically generating burst images
To the function `plot_spectrogram` we can pass a path
```python
from ecallisto_ng.plotting.plotting import plot_spectrogram
plot_spectrogram(df, 'path/to/output/folder')
```
Thus, we fetch the data from the burst list and save the images in the specified folder. The images are saved in the following format: `YYYYMMDD_HHMMSS.png`. The function also returns a list of the paths to the saved images.

In [2]:
from ecallisto_ng.data_download.downloader import (
    get_ecallisto_data,
    get_instrument_with_available_data,
    get_ecallisto_data_generator,
)
import os
from ecallisto_ng.plotting.plotting import plot_spectrogram
from datetime import timedelta

In [3]:
# Some parameters, to make the image creation faster
BUFFER_BEFORE_AFTER = timedelta(minutes=3)
OUTPUT_FOLDER = "burst_images"
TYPES = [3]
INSTRUMENTS_TO_DOWNLOAD = ["Australia_ASSA_02"] # Burst list misses the antenna ID.

## Example
Here is a short example to get started.

In [ ]:
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
# Filter burst list
bl_filtered = burst_list[burst_list.instruments.isin(INSTRUMENTS_TO_DOWNLOAD) & burst_list.type.isin(TYPES)]
# Iterate over all bursts
for i, row in burst_list.iterrows():
    for instrument in get_instrument_with_available_data(row.datetime_start, row.datetime_end):
        if any([x in instrument for x in INSTRUMENTS_TO_DOWNLOAD]): # Burst list misses the antenna ID, thus this is the very ugly way to filter
            df = get_ecallisto_data(
                row.datetime_start - BUFFER_BEFORE_AFTER,
                row.datetime_end + BUFFER_BEFORE_AFTER,
                instrument,
            )
            # Create path to save
            path = os.path.join(OUTPUT_FOLDER, str(row['type']), f"{instrument}_{row.datetime_start.strftime('%Y%m%d_%H%M%S')}.png")
            # Plot the spectrogram
            fig = plot_spectrogram(df, save_path=path)